# Credits

This is heavily influenced by https://github.com/pytorch/tutorials.

# Long Short-Term Memory RNN

This notebook
* Introduces the LSTM
* Combines what we have seen in the previous two notebooks
* Use an LSTM to perform sentiment analysis on SST
__



## Why does
The vanilla RNN has issues with vanishing gradients which give challenges in saving memory over longer sequences.

To battle these issues the gated hidden units were create.
We have Long Short-Term Memory (LSTM) (see [Christopher Olah's walk through](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)) and Gated Recurrent Unit (GRU) which have shown increased performance in saving and reusing memory in later timesteps.

![lstm](../static_files/lstm_cell.png)
source: https://arxiv.org/abs/1412.7828



The LSTM contains three gates, input, forget, output gates and a memory cell.
The output of the LSTM unit is computed with the following functions, where $\sigma = \mathrm{softmax}$.
We have input gate $i$, forget gate $f$, and output gate $o$ defines as

- $i = \sigma ( U^i x_t + W^i h_{t-1})$

- $f = \sigma ( U^f x_t + W^f h_{t-1})$

- $o = \sigma ( U^o x_t + W^o h_{t-1})$

where $U^i, U^f, U^o$ are weight matrices applied to $x_t$ (input vector), and
$W^i, W^f, W^o$ are weight matrices applied to $h_{t-1}$ (hidden state vector) for each respective gate.

$h_{t-1}$, from the previous time step along with the current input $x_t$ are used to compute the a candidate $g$

- $g = \mathrm{tanh}( U^g x_t +  W^g h_{t-1})$

The value of the cell's memory, $c_t$, is updated as

- $c_t = c_{t-1} \circ f + g \circ i$

where $c_{t-1}$ is the previous memory, and $\circ$ refers to element-wise multiplication.

The output, $h_t$, is computed as

- $h_t = \mathrm{tanh}(c_t) \circ o$

and it is used for both the timestep's output and the next timestep, whereas $c_t$ is exclusively sent to the next timestep.
This makes $c_t$ a memory feature, and is not used directly to compute the output of the timestep.

# Stanford sentiment treebank

We will continue with the SST.
See previous notebook for more information.

The code below loads the data, similar to what we did previously.

In [1]:
import numpy as np

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, RNN, LSTM
from torch.nn.functional import softmax, relu

from sklearn.manifold import TSNE

# we'll use the bokeh library to create beautiful plots
# *_notebook functions are needed for correct use in jupyter
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

In [2]:
use_cuda = torch.cuda.is_available()

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

In [84]:
# we assume that all fields are sequential, i.e. there will be a sequence of data
# however, the label field will not contain any sequence
TEXT = data.Field(sequential=True)
LABEL = data.Field(sequential=False)

# create SST dataset splits
# note, we remove samples with neutral labels
train_set, validation_set, _ = datasets.SST.splits(TEXT,
                                                   LABEL,
                                                   fine_grained=False,
                                                   train_subtrees=True,
                                                   filter_pred=lambda ex: ex.label != 'neutral')

# build the vocabularies
# NOTE you should download the GloVe vocabulary, it is quite large..
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
#TEXT.build_vocab(train_set, max_size=None, vectors=Vectors('wiki.simple.vec', url=url))

TEXT.build_vocab(train_set, max_size=None, vectors=[GloVe(name='840B', dim='300')])
LABEL.build_vocab(train_set)
# make iterator for splits
# device gives a CUDA enabled device (-1 disables it)
train_iter, val_iter, _ = data.BucketIterator.splits((train_set, validation_set, _),
                                                     batch_size=128, 
                                                     device=0 if use_cuda else -1)

.vector_cache\glove.840B.300d.zip:  24%|██████████                                | 519M/2.18G [40:10<2:08:25, 215kB/s]


ConnectionAbortedError: [WinError 10053] Une connexion établie a été abandonnée par un logiciel de votre ordinateur hôte

# Build the model

We now use an `RNN` to classify the sentences.
(You will change this into an LSTM as an exercise).



In [79]:
# size of embeddings
embedding_dim = TEXT.vocab.vectors.size()[1]
num_embeddings = TEXT.vocab.vectors.size()[0]
num_classes = len(LABEL.vocab.itos)

print(num_classes)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings, embedding_dim)

        # use pretrained embeddings
        self.embeddings.weight.data.copy_(TEXT.vocab.vectors)
        self.embeddings.weight.detach_()
        
        self.l1 = Linear(in_features=300,
                            out_features=300,
                            bias=False)
        
        self.rnn_1 = LSTM(input_size=embedding_dim,
                         hidden_size=300,
                         num_layers=1,
                         bidirectional=True)
        
        self.l2 = Linear(in_features=1200,
                            out_features=150,
                            bias=False)
        
        self.l3 = Linear(in_features=150,
                            out_features=150,
                            bias=False)
        
        self.l_out = Linear(in_features=150,
                            out_features=num_classes,
                            bias=False)
        
    def forward(self, x):
        out = {}
        # get embeddings
        
        
        x = self.embeddings(x)
        
        x = self.l1(x)

        # rnn returns output and last hidden state
        x, hn = self.rnn_1(x)
        
        # get a fixed sized hidden representation of the entire sequence
        out['hidden'] = x = torch.cat((torch.mean(x, dim=0), torch.max(x, dim=0)[0]), dim=1)
        
        x = nn.Dropout(p=0.5)(relu(self.l2(x)))
        
        x = nn.Dropout(p=0.5)(relu(self.l3(x)))
        
        # classify
        out['out'] = softmax(self.l_out(x), dim=1)
        return out

net = Net()
if use_cuda:
    net.cuda()
print(net)

3
Net(
  (embeddings): Embedding(18005, 300)
  (l1): Linear(in_features=300, out_features=300, bias=False)
  (rnn_1): LSTM(300, 300, bidirectional=True)
  (l2): Linear(in_features=1200, out_features=150, bias=False)
  (l3): Linear(in_features=150, out_features=150, bias=False)
  (l_out): Linear(in_features=150, out_features=3, bias=False)
)


For training we don't want to change the embedding.
We therefore need to filter it out from the parameters that we pass to the optimizer.

In [80]:
# check which params require grad
{p[0]: p[1].requires_grad for p in net.named_parameters()}

{'embeddings.weight': False,
 'l1.weight': True,
 'l2.weight': True,
 'l3.weight': True,
 'l_out.weight': True,
 'rnn_1.bias_hh_l0': True,
 'rnn_1.bias_hh_l0_reverse': True,
 'rnn_1.bias_ih_l0': True,
 'rnn_1.bias_ih_l0_reverse': True,
 'rnn_1.weight_hh_l0': True,
 'rnn_1.weight_hh_l0_reverse': True,
 'rnn_1.weight_ih_l0': True,
 'rnn_1.weight_ih_l0_reverse': True}

In [81]:
criterion = nn.CrossEntropyLoss()
# we filter the model's parameters such that we can remove the embedding layer, 
# which does not have requires_grad
optimizer = optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=5e-4)

def accuracy(ys, ts):
    # making a one-hot encoded vector of correct (1) and incorrect (0) predictions
    correct_prediction = torch.eq(torch.max(ys, 1)[1], ts)
    # averaging the one-hot encoded vector
    return torch.mean(correct_prediction.float())

def construct_sentences(batch):
    return [" ".join([TEXT.vocab.itos[elm] 
                      for elm in get_numpy(batch.text[:,i])])
            for i in range(batch.text.size()[1])]

def get_labels(batch):
    return [LABEL.vocab.itos[get_numpy(batch.label[i])] for i in range(len(batch.label))]

In [82]:
# to project our hidden embeddings to a visualizable space
tsne = TSNE(perplexity=10.0, learning_rate=5.0, n_iter=2000)

# index for each label
colormap = {1: 'DodgerBlue', 2: 'FireBrick'}
# create a tmp source to be updated later
validation_set_size = len(validation_set)
source = ColumnDataSource(data={'x': np.random.randn(validation_set_size),
                                'y': np.random.randn(validation_set_size),
                                'colors': ['green']*validation_set_size,
                                'sentences': ["tmp"]*validation_set_size,
                                'labels': ["unk"]*validation_set_size})
# instance to define hover logic in plot
hover = HoverTool(tooltips=[("Sentence", "@sentences"), ("Label", "@labels")])

# set up the bokeh figure for later visualizations
p = figure(tools=[hover])
p.circle(x='x', y='y', fill_color='colors', size=5, line_color=None, source=source)

def update_plot(meta, layer, handle):
    """ Update existing plot
    
    Parameters
    ----------
    meta: dict
    layer: str
    """
    tsne_acts = tsne.fit_transform(meta[layer])
    source.data['x'] = tsne_acts[:,0]
    source.data['y'] = tsne_acts[:,1]
    source.data['colors'] = [colormap[l] for l in meta['label_idx']]
    
    source.data['sentences'] = meta['sentences']
    source.data['labels'] = meta['labels']
    
    # this updates the given plot
    push_notebook(handle=handle)

## Train the LSTM Model

**Warning** this might take a while.
Go get a cop of coffe, and enjoy the visualizations.

Notice that each data point in the plot corresponds to an entire sentence in the validation set.

In [83]:
max_iter = 25000
eval_every = 1000
log_every = 500
tsne_every = eval_every * 5

# will be updated while iterating
tsne_plot = show(p, notebook_handle=True)

train_loss, train_accs = [], []

net.train()
for i, batch in enumerate(train_iter):
    if i % eval_every == 0:
        net.eval()
        val_losses, val_accs, val_lengths = 0, 0, 0
        val_meta = {'label_idx': [], 'sentences': [], 'labels': []}
        for val_batch in val_iter:
            output = net(val_batch.text)
            # batches sizes might vary, which is why we cannot just mean the batch's loss
            # we multiply the loss and accuracies with the batch's size,
            # to later divide by the total size
            val_losses += criterion(output['out'], val_batch.label) * val_batch.batch_size
            val_accs += accuracy(output['out'], val_batch.label) * val_batch.batch_size
            val_lengths += val_batch.batch_size
            
            for key, _val in output.items():
                if key not in val_meta:
                    val_meta[key] = []
                val_meta[key].append(get_numpy(_val)) 
            val_meta['label_idx'].append(get_numpy(val_batch.label))
            val_meta['sentences'].append(construct_sentences(val_batch))
            val_meta['labels'].append(get_labels(val_batch))
        
        for key, _val in val_meta.items():
            val_meta[key] = np.concatenate(_val)
        
        # divide by the total accumulated batch sizes
        val_losses /= val_lengths
        val_accs /= val_lengths
        
        #print("### EVAL loss: {:.2f} accs: {:.2f}".format(get_numpy(val_losses),get_numpy(val_accs)))
        if i % tsne_every == 0:
            update_plot(val_meta, 'out', tsne_plot)
        
        net.train()
    
    output = net(batch.text)
    batch_loss = criterion(output['out'], batch.label)
    
    train_loss.append(get_numpy(batch_loss))
    train_accs.append(get_numpy(accuracy(output['out'], batch.label)))
    
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
    
    if i % log_every == 0:        
        print("train, it: {} loss: {:.2f} accs: {:.2f}".format(i, 
                                                               np.mean(train_loss), 
                                                               np.mean(train_accs)))
        # reset
        train_loss, train_accs = [], []
        
    
    if max_iter < i:
        break

C:\Python36\lib\site-packages\torchtext\data\field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


train, it: 0 loss: 1.10 accs: 0.47
train, it: 500 loss: 0.82 accs: 0.72
train, it: 1000 loss: 0.76 accs: 0.79
train, it: 1500 loss: 0.75 accs: 0.80
train, it: 2000 loss: 0.73 accs: 0.81
train, it: 2500 loss: 0.73 accs: 0.82
train, it: 3000 loss: 0.72 accs: 0.83
train, it: 3500 loss: 0.71 accs: 0.84
train, it: 4000 loss: 0.70 accs: 0.84
train, it: 4500 loss: 0.70 accs: 0.85
train, it: 5000 loss: 0.69 accs: 0.85
train, it: 5500 loss: 0.69 accs: 0.86
train, it: 6000 loss: 0.68 accs: 0.86
train, it: 6500 loss: 0.68 accs: 0.87


KeyboardInterrupt: 

The above vanilla model should achieve below 80% accuracy in evaluation.

# Assignments

## Assignment 1

Upgrade the model such that it is equivalent to the model presented in [Johansen & Socher](https://arxiv.org/abs/1712.05483).
See figure A3 for an illustration.

- Note, batch and sequence dimensions are flipped
- A *projection layer* is a fully connected layer, which does not necessarily have a non-linearity
- Notice, that you also need to update the optimizer

> **Goal** you should see evaluation acccuracy around 86%

# Optional reading material and lab

- follow [pytorch's seq2seq lab](http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) for a complete implementation of a seq2seq model

also, you might want to read the following articles (which are also mentioned in the above lab):

- [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)
- [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
- [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)
- [A Neural Conversational Model](https://arxiv.org/abs/1506.05869)